# MNS - Biological Plausible Deep Learning
## Run Guergiev et al (2017) Pipeline

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

* **General Structure of Network Module**

    * Init: Declare layer shapes, data, define spike history buffer, init weights/layers
    * init_weights: Loops over all layers and initializes weights
       * Hyperparameters
            - use_weight_optimization: Desired avg/sd of somatic potentials
            - use_feedback_bias: Not in paper - not only synapses Y but also intercept
            - use_sparse_feedback: 80% of weights randomly set to 0
            - use_broadcast: feedback to all layers comes from output layer
        * TODOs
            - Don't loop over layers but sample ones a large set
            - Add dropout percentage and magnitude increase vars
            
    * make_weights_symmetric: 
        - Hyperparameters: Feedback weights = Transposes feedforward weights
            - noisy_symmetric_weights: Add N(0, 0.05) noise to data
            - use_broadcast: feedback to all layers comes from output layer
        - TODOs
            - use_broadcast only needed with more than one hidden layer!
    * init_layers: Loops over layers and adds them to l object in class
    * out_f: Forward phase pass through net - calls out_f of different layers in loop
    * out_t: Target phase pass through net - adds target at top layer
        - TODOs
            - merge out_f and out_t - None vs self.t
    * f_phase: Forward phase
    * t_phase: Target phase
    * train: Loop over epochs and loop over all examples
    * test_weights
    * save_weights
    
* **Layer Module**

    * Init
    * spike
    
* **hiddenLayer**
    
    * Init
    * create_integration_vars
    * clear_vars
    * update_W
    * update_Y
    * update_A
    * update_B
    * update_C
    * out_f
    * out_t
    * plateau_f
    * plateau_t
    * calc_averages
    
* **finalLayer**

    * Init
    * create_integration_vars
    * clear_vars 
    * update_W  
    * update_B
    * update_I
    * update_C
    * out_f
    * out_t
    * calc_averages

In [2]:
import shutil
from sklearn.model_selection import train_test_split

# Import log-helper/learning plot functions
from utils.helpers import *
from utils.plotting import *

# Guergiev et al (2017) - Segregated Compartments DNN Learning
import models.CompDNN as cdl

In [3]:
# Create all necessary directory if non-existent
global data_dir
data_dir = os.getcwd() +"/data"

if not os.path.exists(data_dir):
    os.makedirs(data_dir)
    print("Created New Data Directory")
    
download_data()

No download of MNIST needed.
No download of Fashion-MNIST needed.
No download of CIFAR-10 needed.


In [4]:
# MNIST dataset
X_mnist, y_mnist = get_data(num_samples=70000, dataset="mnist")

X_train, X_test, y_train, y_test = train_test_split(X_mnist, y_mnist,
                                                    stratify=y_mnist,
                                                    random_state=0,
                                                    test_size=1./7)

In [ ]:
train_model("dnn", dnn_model, 10,
                    X_train, y_train, batch_size,
                    device, optimizer, criterion,
                    log_freq = 5000,
                    model_fname ="temp_model_dnn_mnist",
                    verbose=True, logging=True)

In [ ]:
# silence 80% of feedback weights
cdl.use_sparse_feedback = False

# set training parameters
f_etas = (0.21, 0.21)
b_etas = None
n_epochs = 10
n_training_examples = X_train.shape[1]

n=(500, 10)
# create the network -- this will also load the MNIST dataset files
net = cdl.Network(n, X_train, y_train)

# train the network
net.train(f_etas, b_etas, n_epochs)